In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
print("Hello")

Hello


# importing libraries

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision import models
import os
import time

# Building data loaders

In [4]:
dataset_dir = '/kaggle/input/img-dataset-q1/images'  

In [5]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

train_dir = os.path.join(dataset_dir, 'resized_train')
test_dir = os.path.join(dataset_dir, 'resized_test')

# Load the training and testing datasets using ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=transform)
test_data = datasets.ImageFolder(test_dir, transform=transform)

# Create DataLoader instances for training and testing
train_loader = DataLoader(train_data, batch_size=12, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False)

# 1 block VGG

In [6]:
class SimpleVGGBlock(nn.Module):
    def __init__(self):
        super(SimpleVGGBlock, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 224 * 224, 2)  # Output layer with 2 units for binary classification

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc1(x)  # Output layer with 2 units
        return x

### Model definition

In [7]:
model = SimpleVGGBlock()
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification (binary classification with 2 classes)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
train_loss = []
train_acc = []

### Training and inference

In [8]:
start_time = time.time()
#Training 
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  
        
        running_loss += loss.item()  
        
        _, predicted = torch.max(outputs, 1) 
        total += labels.size(0) 
        correct += (predicted == labels).sum().item() 
        
    accuracy = 100 * correct / total
    train_loss.append(running_loss/len(train_loader))
    train_acc.append(accuracy)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")
end_time = time.time()

print(f'\nTime taken to train the model is {end_time - start_time}')

Epoch [1/10], Loss: 8.9676, Accuracy: 58.12%
Epoch [2/10], Loss: 1.1181, Accuracy: 80.00%
Epoch [3/10], Loss: 0.8018, Accuracy: 81.25%
Epoch [4/10], Loss: 0.7652, Accuracy: 84.38%
Epoch [5/10], Loss: 0.1265, Accuracy: 94.38%
Epoch [6/10], Loss: 0.0381, Accuracy: 98.75%
Epoch [7/10], Loss: 0.0080, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0073, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0027, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0018, Accuracy: 100.00%

Time taken to train the model is 66.90767645835876


In [9]:
model.eval()  
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)  
        _, predicted = torch.max(outputs, 1) 
        total += labels.size(0)  
        correct += (predicted == labels).sum().item()  

print(f"\nTraining loss history: {train_loss}")
print(f"\nTraining accuracy history: {train_acc}")
# Test accuracy
test_accuracy = 100 * correct / total
print(f"\nTest Accuracy: {test_accuracy:.2f}%")

# Number of model parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nNumber of model parameters: {total_params}")


Training loss history: [8.967631974390574, 1.1180523135034102, 0.8018220148182341, 0.7651546373318914, 0.12650143319375015, 0.03806039148080994, 0.008049304562778812, 0.007329075855003404, 0.0026956598875195986, 0.0017750315689357063]

Training accuracy history: [58.125, 80.0, 81.25, 84.375, 94.375, 98.75, 100.0, 100.0, 100.0, 100.0]

Test Accuracy: 70.00%

Number of model parameters: 3216354


# 3 block VGG 

In [12]:
class VGG3Block(nn.Module):
    def __init__(self):
        super(VGG3Block, self).__init__()
        
        # 3 convolutional blocks with Conv2d and MaxPool2d
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool
        )
        
        self.fc1 = nn.Linear(256 * 28 * 28, 512)  
        self.fc2 = nn.Linear(512, 2)  # Output layer with 2 units (binary classification)

    def forward(self, x):
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        
        x = x.view(x.size(0), -1)
        
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  
        
        return x

In [11]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [13]:
model_two_3blocks = VGG3Block().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model_two_3blocks.parameters(), lr=0.0001)
train_loss = []
train_acc = []

In [14]:
start_time = time.time()

num_epochs = 10
for epoch in range(num_epochs):
    model_two_3blocks.train() 
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()  
        outputs = model_two_3blocks(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  
        
        running_loss += loss.item() 
        
        _, predicted = torch.max(outputs, 1)  
        total += labels.size(0)  
        correct += (predicted == labels).sum().item()  
    
    accuracy = 100 * correct / total
    train_loss.append(running_loss/len(train_loader))
    train_acc.append(accuracy)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

end_time = time.time()
print(f'\nTime taken to train the model_two_3blocks is {end_time - start_time:.2f} seconds')

Epoch [1/10], Loss: 0.7019, Accuracy: 51.88%
Epoch [2/10], Loss: 0.6520, Accuracy: 64.38%
Epoch [3/10], Loss: 0.5789, Accuracy: 71.88%
Epoch [4/10], Loss: 0.5168, Accuracy: 70.62%
Epoch [5/10], Loss: 0.4011, Accuracy: 80.00%
Epoch [6/10], Loss: 0.3033, Accuracy: 88.75%
Epoch [7/10], Loss: 0.1879, Accuracy: 93.75%
Epoch [8/10], Loss: 0.1174, Accuracy: 94.38%
Epoch [9/10], Loss: 0.1584, Accuracy: 93.12%
Epoch [10/10], Loss: 0.1953, Accuracy: 89.38%

Time taken to train the model_two_3blocks is 14.27 seconds


In [17]:
model_two_3blocks.eval()  # Set the model_two_3blocks to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_two_3blocks(inputs)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted class labels
        total += labels.size(0)  # Update total samples
        correct += (predicted == labels).sum().item()  
test_accuracy = 100 * correct / total

total_params = sum(p.numel() for p in model_two_3blocks.parameters() if p.requires_grad)

print(f"\nTraining loss history: \n{train_loss}")
print(f"\nTraining accuracy history:\n {train_acc}")
print(f"\nTest Accuracy: {test_accuracy:.2f}%")
print(f"\nNumber of model_two_3blocks parameters: {total_params}")


Training loss history: 
[0.7019052675792149, 0.6520498096942902, 0.5789400977747781, 0.516767544405801, 0.4010657998067992, 0.30333614881549564, 0.18785322776862554, 0.11743209644087724, 0.15843702041144883, 0.19533697622162954]

Training accuracy history:
 [51.875, 64.375, 71.875, 70.625, 80.0, 88.75, 93.75, 94.375, 93.125, 89.375]

Test Accuracy: 65.00%

Number of model_two_3blocks parameters: 103907394


# 3 block with image augmentation 

In [19]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(20), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # VGG normalization
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # VGG normalization
])


train_dir = os.path.join(dataset_dir, 'resized_train')
test_dir = os.path.join(dataset_dir, 'resized_test')

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False)

class VGGBlock(nn.Module):
    def __init__(self):
        super(VGGBlock, self).__init__()
        
        # 3 convolutional blocks with Conv2d and MaxPool2d
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Conv1
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool
        )
        
        # Fully connected layer for binary classification
        self.fc1 = nn.Linear(256 * 28 * 28, 512)  # Linear layer after flattening the output of the conv layers
        self.fc2 = nn.Linear(512, 2)  # Output layer with 2 units (binary classification)

    def forward(self, x):
        # Pass the input through each block
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        
        # Flatten the output
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # Output layer
        
        return x

In [22]:
model_data_aug = VGGBlock().to(device)  # Move model_data_aug to device (CPU or GPU)
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification (binary classification with 2 classes)
optimizer = optim.Adam(model_data_aug.parameters(), lr=0.0001)

# Lists to track loss and accuracy during training
train_loss = []
train_acc = []

start_time = time.time()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model_data_aug.train()  # Set the model_data_aug to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        # Move data to device (CPU or GPU)
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the gradients
        
        # Forward pass
        outputs = model_data_aug(inputs)  
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        
        running_loss += loss.item()  # Accumulate loss
        
        # Get predicted class labels
        _, predicted = torch.max(outputs, 1)  # Get the index of the max output (class prediction)
        
        total += labels.size(0)  # Update total samples
        correct += (predicted == labels).sum().item()  # Update correct predictions
        
        # Print loss and accuracy after every few batches
        if (i + 1) % 10 == 0:
            batch_accuracy = 100 * correct / total
            print(f"Batch [{i+1}/{len(train_loader)}] Loss: {running_loss / (i+1):.4f}, Accuracy: {batch_accuracy:.2f}%")
    
    # Calculate accuracy for the epoch
    accuracy = 100 * correct / total
    train_loss.append(running_loss / len(train_loader))
    train_acc.append(accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

end_time = time.time()
print(f'\nTime taken to train the model_data_aug: {end_time - start_time} seconds')


Batch [10/20] Loss: 0.7000, Accuracy: 56.25%
Batch [20/20] Loss: 0.7056, Accuracy: 52.50%
Epoch [1/10], Loss: 0.7056, Accuracy: 52.50%
Batch [10/20] Loss: 0.6899, Accuracy: 57.50%
Batch [20/20] Loss: 0.6981, Accuracy: 50.00%
Epoch [2/10], Loss: 0.6981, Accuracy: 50.00%
Batch [10/20] Loss: 0.6948, Accuracy: 47.50%
Batch [20/20] Loss: 0.6945, Accuracy: 46.88%
Epoch [3/10], Loss: 0.6945, Accuracy: 46.88%
Batch [10/20] Loss: 0.6939, Accuracy: 46.25%
Batch [20/20] Loss: 0.6914, Accuracy: 50.00%
Epoch [4/10], Loss: 0.6914, Accuracy: 50.00%
Batch [10/20] Loss: 0.6695, Accuracy: 57.50%
Batch [20/20] Loss: 0.6822, Accuracy: 55.62%
Epoch [5/10], Loss: 0.6822, Accuracy: 55.62%
Batch [10/20] Loss: 0.6995, Accuracy: 50.00%
Batch [20/20] Loss: 0.6843, Accuracy: 54.38%
Epoch [6/10], Loss: 0.6843, Accuracy: 54.38%
Batch [10/20] Loss: 0.6698, Accuracy: 66.25%
Batch [20/20] Loss: 0.6737, Accuracy: 61.88%
Epoch [7/10], Loss: 0.6737, Accuracy: 61.88%
Batch [10/20] Loss: 0.6112, Accuracy: 67.50%
Batch [20/

In [23]:

model_data_aug.eval()  
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        # Move data to device (CPU or GPU)
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model_data_aug(inputs)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted class labels
        total += labels.size(0)  # Update total samples
        correct += (predicted == labels).sum().item()  # Update correct predictions

test_accuracy = 100 * correct / total
print(f"\nTest Accuracy: {test_accuracy:.2f}%")

# Number of model_data_aug parameters
total_params = sum(p.numel() for p in model_data_aug.parameters() if p.requires_grad)
print(f"\nNumber of model_data_aug parameters: {total_params}")


Test Accuracy: 72.50%

Number of model_data_aug parameters: 103907394


# testing ai images 

In [33]:
from PIL import Image
import numpy as np

In [34]:
folder_path = '/kaggle/input/img-generated-ai'
label_mapping = {
    "antelope": 0,
    "rabbit": 1
}

# Initialize empty lists for images, labels, and file names
test_images = []
test_labels = []
file_names = []

# Iterate through the files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(('.png', '.jpg', '.jpeg')):  # Ensure it's an image
        # Load the image
        image_path = os.path.join(folder_path, file_name)
        image = Image.open(image_path).convert("RGB")  # Convert to RGB format
        
        # Resize the image (optional, standardize to a consistent size, e.g., 224x224)
        image = image.resize((224, 224))
        
        # Convert the image to a numpy array
        image_array = np.array(image)
        test_images.append(image_array)
        
        # Store the file name
        file_names.append(file_name)
        
        # Determine the label based on the file name
        if "antelope" in file_name.lower():
            test_labels.append(label_mapping["antelope"])
        elif "rabbit" in file_name.lower():
            test_labels.append(label_mapping["rabbit"])

# Convert lists to numpy arrays for easy handling
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Print out the dataset shape and file names for verification
print(f"Test Images Shape: {test_images.shape}")
print(f"Test Labels Shape: {test_labels.shape}")
print(f"File Names: {file_names}")
print(f"Labels: {test_labels}")

Test Images Shape: (4, 224, 224, 3)
Test Labels Shape: (4,)
File Names: ['easy_rabbit.jpg', 'easy_antelope.jpg', 'hard_rabbit.jpg', 'hard_antelope.jpg']
Labels: [1 0 1 0]


In [35]:
from torch.utils.data import DataLoader, Dataset

In [36]:
# Define a custom Dataset class for the test set
class CustomTestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label

In [47]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize (use your model's normalization parameters)
])

# Create the test dataset and DataLoader
test_dataset_gen = CustomTestDataset(test_images, test_labels, transform=transform)
test_loader_gen = DataLoader(test_dataset_gen, batch_size=4, shuffle=False)

In [48]:
def test_fnai(model_gen):    
    model_gen.eval()  # Set the model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_gen = model_gen.to(device)  # Move model to device
    
    # Inference loop
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():  # No need to compute gradients during inference
        for images, labels in test_loader_gen:
            images = images.to(device)  # Move images to the same device as the model
            labels = labels.to(device)  # Move labels to the same device
    
            # Perform inference
            outputs = model_gen(images)
            predictions = torch.round(torch.sigmoid(outputs)).squeeze()  # Apply sigmoid and round for binary classification
    
            # Store predictions and labels
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Print predictions and corresponding ground truth
    for idx, (pred, label) in enumerate(zip(all_predictions, all_labels)):
        print(f"Image {idx + 1}: Predicted: {int(pred)}, Ground Truth: {int(label)}")